# Funkce vyššího řádu, dekorátory a modul functools

Už minule jsme si řekli něco málo o funkcích vyššího řádu a ukázali jsme si, jak je možné funkce předávat jiným funkcím jako argumenty.

Dnes si ukážeme, jak funkce generovat v rámci jiných funkcí, jak s tím souvisí dekorátory a dojde i na slíbenou poznámku o funkcionálním programování.

## Opakování

Funkce vyššího řádu jsou takové, které:
* mohou brát jiné funkce jako své argumenty nebo
* mohou vracet nové funkce jako svou návratovou hodnotu.

Takto pracovat s funkcemi je možné díky tomu, že funkce je v Pythonu objekt jako každé jiný - jako jakákoli jiná proměnná.

## Funkce jako argument jiné funkce

Minule jsme si ukázali několik funkcí, které jako argument braly jinou funkci a chovaly se podle jejich výsledků - `map`, `reduce` a `filter`.

Dnes si na příkladu ukážeme, jak takovou funkci sami napsat.

Nejdříve si definujeme dvě obyčejné funkce pro základní matematické operace:

In [1]:
def secti(cisla):
    """Funkce pro sečtení sekvence čísel"""
    vysledek = 0
    for cislo in cisla:
        vysledek += cislo
    return vysledek

In [2]:
def vynasob(cisla):
    """Funkce pro vynásobení sekvence čísel"""
    vysledek = 1
    for cislo in cisla:
        vysledek = vysledek * cislo
    return vysledek

A teď jednu obecnou funkci, které bude umět provést jakoukoli funkci se zadaným argumentem a vrátit její výsledek:

In [3]:
def proved(funkce, argument):
    """Funkce volající jinou funkci s jedním argumentem"""
    return funkce(argument)

Nezbývá než vyzkoušet, zda je možné použít obyčejné funkce `secti` a `vynasob` jako první argument pro funkci vyššího řádu `proved`.

In [4]:
proved(secti, [1, 2, 3, 4])

10

In [5]:
proved(vynasob, [1, 2, 3, 4])

24

Stále platí, že předáváme-li funkci (např. `secti`) jako argument do jiné funkce (např. `proved`), není třeba ji definovat jako pojmenovanou, ale můžeme využít anonymní funkce. Naše funkce `proved` je natolik obecná, že s ní dokážeme udělat takřka cokoliv.

In [6]:
proved(lambda x: " ".join(x), ["Pepa", "nesl", "tašku"])

'Pepa nesl tašku'

Jako první argument jsme definovali anonymní funkci, která jednoduše spojí řetězce pomocí mezery do jednoho. Druhý argument pak obsahuje seznam tří řetězců.

Tento koncept už známe z minula. Teď je na čase se na něj podívat z druhé strany.

## Funkce generující jinou funkci

Protože funkce je objekt jako každý jiný, může být vytvořena v rámci jiné funkce a vrácena jako její návratová hodnota.

Nejdříve si to opět ukážeme na jednoduchém příkladu. Chceme si napsat funkci, která nám bude umět definovat a vrátit novou funkci, která k argumentem zadanému číslu bude přičítat nějaké pevně dané číslo.

In [7]:
def funkce_pricitani(kolik):
    """Vnější funkce, která generuje novou funkci uvnitř"""
    
    def pricitani(k_cemu):
        """Vnitřní funkce, která bude definována ve chvíli volání nější funkce a vrácena"""
        return kolik + k_cemu

    return pricitani

Funkce `funkce_pricitani` bere jeden argument - číslo, které se má při volání nově vygenerované funkce přičíst k zadanému argumentu - a vrátí novou funkci, které přesně takové přičítání bude umět. Nově definovaná funkce pak bere také jeden argument, který zadáme při jejím volání.

Nejdříve si takto můžeme generovat funkci, která už bude mít inkrement definován. Například pro pětku a dvacítku.

In [8]:
pricti_5 = funkce_pricitani(5)

In [9]:
pricti_20 = funkce_pricitani(20)

Do proměnných `pricti_5` a `pricti_20` se nám tak uložily dvě různé funkce, které lze volat a jedním argumentem.

In [10]:
pricti_5(1)

6

In [11]:
pricti_20(2)

22

Za povšimnutí stojí, že kromě definice se v tomto příkladu nikde nepoužívá originální název vnitřní funkce `pricitani`, který je stejně jako u jakékoli jiné funkce lokální proměnnou a na konci nadřazené funkce se na něj zapomene.

Pojďme na trošku realističtější případ použití. Výpočet daně z nemovitosti se řídí cenou za m², celkovou plochou a také koeficientem města či obce. Můžete mít funkci, která tyto argumenty bude potřebovat při každém spuštění, nebo si můžete vytvořit funkci pro typ pozemku v daném městě a do ní už doplnit jen jeho rozlohu. My půjdeme tou druhou cestou.

In [12]:
def dan_z_nemovitosti(koeficient, cena_za_m2):
    """Funkce definuje a vráti funkci pro výpočet daně z nemovitosti v dané obci"""
    
    def vypocet_dane(plocha):
        """Vypočte daň z nemovitosti v dané lokalitě"""
        return plocha * cena_za_m2 * koeficient
    
    return vypocet_dane

Díky takto dynamicky definované funkci si následně můžeme vytvořit funkci na výpočet daně z rodinného domu v Ostravě, kde koeficient je 3,5 a cena za m² je 2 Kč.

In [13]:
dum_ostrava = dan_z_nemovitosti(3.5, 2)

Pak už stačí pro každý výpočet daně z rodinného domu v Ostravě použít tuto dynamicky definovanou funkci a jako argument ji zadat jen zastavěnou plochu.

In [14]:
dum_ostrava(125)

875.0

Pro zastavěnou plochu v Praze, kde koeficient je 4,5 a cena za m² je 20 haléřů, bude vypadat velmi podobně.

In [15]:
pozemek_praha = dan_z_nemovitosti(4.5, 0.2)

In [16]:
pozemek_praha(900)

810.0

Takto je možné si jakkoli vytvářet dynamicky definované funkce třeba i na základě vstupu od uživatele či dat z jiného zdroje.

## Dekorátory

Dekorátory jsou interně velmi podobné dynamicky definovaným funkcím, ale mají přece jen jednu odlišnost: Zatímco v předchozích příkladech jsme definovali jednu funkci uvnitř druhé, u dekorátorů definujeme samotný dekorátor mimo vnitřní funkci v obecnější podobě a jen jej na ni aplikujeme. Ve výsledku to znamená, že můžeme dekorátory použít aniž bychom měnili kód původních funkcí.

Opět jeden jednoduchý příklad pro lepší pochopení. Mějme obyčejnou jednoduchou funkci pro sčítání:

In [17]:
def secti(a, b):
    return a + b

Její výsledek závisí na tom, jaké argumenty dostane. Pokud budou obě čísla celá, bude i výsledek celé číslo. Pokud ale bude jedno z čísel s desetinnou tečkou, bude i výsledek typu `float`.

In [18]:
secti(5, 3)

8

In [19]:
secti(5.0, 2)

7.0

Nyní přijde řada na dekorátor, který bude umět manipulovat s výsledkem funkce, na kterou bude aplikován. V našem případě bude umět zajistit, aby výsledek funkce byl vždy typu `float`.

In [20]:
def desetinne_cislo(puvodni_funkce):
    
    def nova_funkce(*args, **kwargs):
        vysledek = puvodni_funkce(*args, **kwargs)
        return float(vysledek)

    return nova_funkce

Dekorátor je sám funkcí, která jako argument bere funkci, nad kterou bude dekorátor postaven. Uvnitř je pak definována nová funkce, která může libovolně pracovat s funkcí původní a obohatit ji tak o jakoukoli funkcionalitu.

Uvnitř dekorátoru jsou pro definici nové funkce a volání té původní použity argumenty v obecném tvaru s `*args` a `**kwargs`, což nám zajistí, že dekorátor bude schopen předávat všechny poziční i pojmenované argumenty.

Takto definovaný dekorátor je možné použít pomocí znaku `@` nad definicí funkce. Například naše funkce `secti` s dekorátorem zajišťujícím konzistenci výsledků bude vypadat takto:

In [21]:
@desetinne_cislo
def secti(a, b):
    return a + b

Takto definovaná funkce s použitým dekorátorem bude vždy vracet desetinné číslo.

In [22]:
secti(5.0, 2)

7.0

In [23]:
secti(5, 3)

8.0

Použití dekorátoru nad definicí funkce pomocí `@` není žádná magie. Jedná se jen o tzv. syntaktický cukr, který usnadňuje zápis a umožňuje velmi snadno použít i více dekorátorů. Bez tohoto cukru bychom mohli dekorátor použít i takto:

In [24]:
def secti(a, b):
    return a + b

In [25]:
secti_float = desetinne_cislo(secti)

In [26]:
secti_float(1, 2)

3.0

Tento zůsob manuálního použití děkorátoru má tu výhodu, že původní funkce je stále dostupná pod svým originálním jménem.

Funkce funguje samozřejmě pouze v případě, že se dá výsledek na desetinné číslo převést. I tato situace by se však dala bez problému v dekorátoru ošetřit.

In [27]:
secti_float("Yet", "ti")

ValueError: could not convert string to float: 'Yetti'

Dalším velmi populárním příkladem použití dekorátorů je měření času trvání funkce a také logování jejího volání.

Například:

In [28]:
def zmer_cas(puvodni_funkce):
    
    def nova_funkce(*args, **kwargs):
        import time
        start = time.time()
        print(f"Volám funkci {puvodni_funkce} s argumenty {args}")
        vysledek = puvodni_funkce(*args, **kwargs)
        end = time.time()
        print(f"Funkce běžela {end - start} sekund")
        return vysledek

    return nova_funkce

Takový dekorátor pak můžeme jednoduše použít na jakoukoli funkci a zjistit, kdy byla volána a jak dlouho její volání trvalo.

In [29]:
@zmer_cas
def mocnina_dvojky(mocnitel):
    return 2 ** mocnitel

In [30]:
mocnina_dvojky(10)

Volám funkci <function mocnina_dvojky at 0x7f84a02284c0> s argumenty (10,)
Funkce běžela 0.000118255615234375 sekund


1024

In [31]:
vysledek = mocnina_dvojky(10000000)

Volám funkci <function mocnina_dvojky at 0x7f84a02284c0> s argumenty (10000000,)
Funkce běžela 0.038305044174194336 sekund


## Modul functools

Dekorátorů je ve standardní knihovně celá řada a jsou hojně využívány i v populárních knihovnách, protože jejich použití může i zdánlivě složité implementace zjednodušit a zpřehlednit.

Pojďme lehce nakouknout do obsahu modulu `functools`.

### @lru_cache

Dekorátor `@functools.lru_cache` aktivuje paměť volání funkce a jejich výsledků. Hodí se především ve chvíli, kdy zpracování výsledků ve funkci trvá dlouho a je lepší si výsledky automaticky zapamatovat a vrátit je z paměti při dalším volání funkce se stejnými argumenty. To se může hodit například potřebujeme-li získat z internetu vícekrát stejnou informaci.

Ukážeme si to na výpočtu čísel z Fibonacciho posloupnosti pomocí rekurze, kdy funkce volá samu sebe, a zapamatování si předchozích výsledků dává smysl.

In [32]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

Funkci několikrát zavoláme. Interně se navíc provede mnohokrát, protože při vyšších číslech se rekurzivně volá sama.

In [33]:
[fib(n) for n in range(16)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610]

Následně je možné si u takto definované funkce zjistit statistiky paměti a úspěšnost její použití při opakovaných voláních.

In [34]:
fib.cache_info()

CacheInfo(hits=28, misses=16, maxsize=None, currsize=16)

`maxsize` a `currsize` označují maximální a aktuální velikost paměti výsledků, `hits` označuje, kolikrát byl při volání funkce použit výsledek z paměti místo opětovného spuštění funkce a `misses` naopak kolikrát musela být funkce volána, protože výsledek v paměti nebyl k dispozici.

### partial

Funkce `partial` není dekorátorem, ale funkcí vyššího řádu, která umí předvyplnit argumenty do již existující funkce tak, aby její volání už je nemuselo obsahovat pokaždé všechny.

Můžeme si tak jednoduše vytvořit vlastní `print`, který se bude chovat podle našich představ:

In [35]:
from functools import partial

muj_print = partial(print, end="!!!", sep="_")

muj_print("Léto", "je", "tady")

Léto_je_tady!!!

Funkci `partial` předáme jako první argument funkci, ze které chceme vycházet, a jako další argumenty pak poziční a pojmenované argumenty, které se mají do původní funkce automaticky doplnit. Takto vytvořenou funkci pak můžeme jednoduše používat, aniž bychom museli opakující se argumenty neustále opakovat.

Ti pozornější z vás si všimnou, že tímto způsobem bychom mohli elegantně vyřešit i naše funkce k výpočtu daní z nemovitosti.

### singledispatch

Dekorátor `singledispatch` nám umožní mít definovány různé funkce pod stejným jménem a volat vždy jednu z nich podle typu prvního argumentu, který při volání předáme. Máte jednu funkci, která by se měla chovat jinak pro čísla, jinak pro řetězce a úplně jinak pro seznamy, ale pořád je to jedna funkce a dává smysl, aby měla jen jedno jméno? Pak je `singledispatch` to pravé.

Nejdříve si definujeme základní funkci, která se zavolá, nebude-li žádná jiná vhodnější.

In [36]:
from functools import singledispatch

@singledispatch
def funkce(argument):
    print(f"Zadal jsi {argument}")

Další funkce můžeme k původní funkci registrovat pomocí dekorátoru, který nám tímto vznikl. Jméno u dalších funkcí není podstatné, takže se nejčastěji nahrazuje podtržítkem.

In [37]:
@funkce.register(int)
def _(argument):
    print(f"Celé číslo děleno dvěma je {argument / 2}")

A v podobném duchu můžeme definovat další funkce pro další typy proměnných a všechny je schovat pod jedno jméno.

In [38]:
@funkce.register(list)
def _(argument):
    soucet = sum(argument)
    print(f"Součet prvků v seznamu je {soucet}")

Pokud pak budeme funkci volat, zavolá se vždy ta správná podle typu prvního argumentu.

In [39]:
funkce(5)

Celé číslo děleno dvěma je 2.5


In [40]:
funkce([1, 2, 3])

Součet prvků v seznamu je 6


In [41]:
funkce("Pepa")

Zadal jsi Pepa


První definovaná funkce se zavolá vždy, když neexistuje žádná jiná, které by typ prvního argumentu odpovídal.

In [42]:
funkce({"jméno": "Karel"})

Zadal jsi {'jméno': 'Karel'}


Podobná situace by se dala řešit i podmínkami uvnitř funkce, ale v případě složitější logiky může `@singledispatch` ušetřit několik úrovní odsazení kódu a ubrat na komplexnosti.

## Funkcionální programování

### Teorie

Funkcionální programování je deklarativní programovací technikou, která je založena na funkcích bez vedlejších efektů, které jen produkují výsledky a spolu s výrazy se rekurzivně vyhodnocují až se dojde k výsledku. U funkcionálního programování se neukládá stav programu do proměnných (objektů), se kterým by pak funkce manipulovaly. Čistě funkcionální jazyky také neobsahují cykly ani podmínky (bloky).

Definice není moc záživná a tak si za chvíli ukážeme nějaký příklad. Podstatné je ale mít na paměti, že i když nebudete programovat čistě funkcionálně, je dobré se z některých přístupů poučit a klidně je používat i v objektově orientovaném programování.

Funkcionální programování se mimo jiné vyznačuje těmito vlastnostmi:
* Funkce vrací výsledky a nemají žádné vedlejší efekty (nemění stav programu)
* Výsledek funkce je závislý jen a pouze na vstupních argumentech
* Nepoužívají se bloky s podmínkami ani cykly
* Hojně se využívají rekurze a anonymní funkce

### Příklady

Bloků s podmínkami se můžeme zbavit pomocí pravdivostních výrazů.

In [43]:
def sude_liche(cislo):
    if cislo % 2 == 0:
        return "sudé"
    else:
        return "liché"

Funkci `sude_liche` je možné přepsat na jednořádkový výraz a případně z něj udělat funkci pomocí lambdy.

In [44]:
sude_liche_2 = lambda x: (x % 2 == 0 and 'sudé') or ('liché')

Výsledky však zůstanou totožné.

In [45]:
sude_liche(3)

'liché'

In [46]:
sude_liche_2(3)

'liché'

Cyklu `for` se můžeme jednoduše zbavit definicí funkce a její aplikace na všechny prvky sekvence pomocí `map` nebo pomocí list comprehensions.

Cyklu `while` se nejsnáze zbavíme pomocí rekurze.

In [47]:
n = 1
while n < 5:
    n += 1
    print(n)

2
3
4
5


In [48]:
def get_n(n=1):
    n += 1
    print(n)
    return n if n >= 5 else get_n(n)

get_n()

2
3
4
5


5

Příklady jsou jen ukázkou toho, jak se jednotlivé programovací techniky navzájem liší a jak je možné jedním jazykem napsat stejný program několika způsoby. Existují samozřejmě jazyky vyloženě určené pro funkcionální programování, u kterých tolik netrpí čitelnost kódu, ale jejich využití je zase do značné míry omezeno jinde.

Další studium nechám na zájemcích.

# Úkol

Úkolem po této lekci je:
1. Znovu si projít celou lekci a zeptat se na případné nejasnosti
2. Implementovat vlastní dekorátor, který najde uplatnění v některé z vašich aplikací nebo bude natolik obecný, že bude užitečný například pro ladění programu

Úkoly a případné dotazy očekávám na emailu frenzy.madness@gmail.com